In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [15]:
df = pd.read_csv("simulated_energy_data.csv")

In [16]:
features = [
    "home_size_sqft","residents","ac_level","climate","time_usage_type",
    "house_type","fridge","washer","dryer","dishwasher","ev_charger","pool_pump"
]
target = "total_kwh"

X = df[features]
y = df[target]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
categorical_features = ["ac_level","climate","time_usage_type","house_type"]
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(), categorical_features)
], remainder="passthrough")

In [19]:
lr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])
lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)


In [20]:
def evaluate(y_true, y_pred, label="Model"):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"--- {label} Metrics ---")
    print(f"R² Score : {r2:.3f}")
    print(f"RMSE     : {rmse:.2f}")
    print(f"MAE      : {mae:.2f}\n")

evaluate(y_test, y_pred, label="Linear Regression")


--- Linear Regression Metrics ---
R² Score : 0.902
RMSE     : 80.68
MAE      : 63.20



In [21]:
utility_rates = {
    "PG&E": {"peak": 0.35, "offpeak": 0.15, "fixed": 15},
    "SCE": {"peak": 0.32, "offpeak": 0.14, "fixed": 12},
    "SDG&E": {"peak": 0.38, "offpeak": 0.16, "fixed": 18}
}

def calculate_bill(predicted_kwh, utility):
    peak_kwh = predicted_kwh * 0.5
    offpeak_kwh = predicted_kwh * 0.5
    rates = utility_rates[utility]
    return round(peak_kwh*rates["peak"] + offpeak_kwh*rates["offpeak"] + rates["fixed"],2)
